#### Overlap testing

In order to address the problems with double-counting, I need to remove the areas of overlap between Natura 2000 sites. In this file, I perform some testing to make sure I remove the overlaps correctly.

My plan for removing the overlaps is basically to dissolve all the polygon boundaries to create to make one big polygon with no overlaps. I then 'explode' the geometry so that the massive multi-part geomtry is separate into individual polygons for each part. This step is needed because the big dissolved multi-part polygon is too big to calculate zonal statistics.

My worry with this approach is that because I would end up creating smaller polygons than in the original dataset (the original dataset includes some multi-part Natura sites), this may mean the area gets calculated differently. In theory I think it should not matter, but I would like to check.

To check for this issue, I do the following steps:
1) Create a test site which is a multi-part polygon with some areas/parts that are very small (small enough not to return results) -- I do this part manually in QGIS
2) Calculate the zonal statistics for this test site in it's original form
3) Separate ('explode') the test site into individual polygons
4) Calculate the zonal statistics for the separated/exploded geometries (and sum them up)
5) Compare the two sets of zonal statistics

My hope is that these would return the same results, even though the second exploded geometry set might return some empty results for the smaller polygons. I think these would also be missing from the result for the multi-part geometry set, but it just wouldn't be clear from the results because you only get a single result back for the whole multi-part polygon. If the results match then I think the dissolve/explode approach to remove the overlaps is a good way forward. 

In [21]:
# SETUP
import pickle
import pandas as pd
from rasterstats import zonal_stats
import geopandas as gpd

# Because I'm working in the 'other' folder, I need to store the main directory info
main_dir = "C:/Users/ninam/Documents/UZH/04_Thesis/code/nm_forest_thesis/"

In [9]:
# CALC MULTIPART STATS

# Load test site (multi-part geom with 1 main part and 2 small parts)
test_site = gpd.read_file(main_dir + "processing/test_site_multipart.shp")

# Load in ESA FNF (for testing)
esa_FNF = main_dir + "outputs/esa_lccs_class_3035_DE_5m_2018_FNF.tif"

# Calculate the zonal stats (count only - just need the total area)
test_site_multipart_stats = zonal_stats(test_site, esa_FNF, stats=['count'],
                                        geojson_out=True)

In [ ]:
# FORMAT RESULTS

# Load as feature collection
test_site_multipart_stats_fc = {"type": "FeatureCollection","features": test_site_multipart_stats}

# Extract features
test_site_multipart_stats_features = test_site_multipart_stats_fc["features"]

# Check results
#test_site_multipart_stats_features

In [12]:
# EXTRACT & CALC TOTAL AREA (MULTIPART)

# Create empty lists for storing the area counts
multipart_count_list = []

# Fill the list with all the area counts
for row in test_site_multipart_stats_features:
    multipart_count_list.append(row['properties']['count'])

# Sum all the values and convert to area (25m2 pixel size = 0.0025 ha)
multipart_total_area_ha = sum(multipart_count_list) * 0.0025

# Show the result
print("Total area (ha) for single multipart geom = ", multipart_total_area_ha)

Total area (ha) for single multipart geom =  22.915


In [13]:
# CREATE EXPLODED VERSION & CALC STATS

# Explode the multipart
test_site_exp = test_site.explode()

# Calculate the zonal stats (count only - just need the total area)
test_site_exp_stats = zonal_stats(test_site_exp, esa_FNF, stats=['count'],
                                  geojson_out=True)

In [15]:
# FORMAT RESULTS

# Load as feature collection
test_site_exp_stats_fc = {"type": "FeatureCollection","features": test_site_exp_stats}

# Extract features
test_site_exp_stats_features = test_site_exp_stats_fc["features"]

# Check results
test_site_exp_stats_features

[{'id': '0',
  'type': 'Feature',
  'properties': {'id': 1, 'count': 0},
  'geometry': {'type': 'Polygon',
   'coordinates': (((4415740.052809142, 3132526.347389419),
     (4415741.647363373, 3132525.721678265),
     (4415741.647363373, 3132525.721678265),
     (4415742.515285296, 3132523.8041763413),
     (4415741.869389911, 3132522.1894378792),
     (4415739.972072219, 3132521.3618844175),
     (4415738.357333756, 3132522.007779802),
     (4415737.549964526, 3132523.8647290333),
     (4415738.074754526, 3132525.4794674953),
     (4415740.052809142, 3132526.347389419)),)},
  'bbox': (4415737.549964526,
   3132521.3618844175,
   4415742.515285296,
   3132526.347389419)},
 {'id': '0',
  'type': 'Feature',
  'properties': {'id': 1, 'count': 4},
  'geometry': {'type': 'Polygon',
   'coordinates': (((4431109.052754132, 3131860.590721509),
     (4431119.104501058, 3131860.954037663),
     (4431119.306343365, 3131850.98302766),
     (4431109.294964901, 3131850.660079967),
     (4431109.05275

The printout above shows that the first geometry does not return any results (count = 0) but the other geoms do.

In [16]:
# EXTRACT & CALC TOTAL AREA (MULTIPART)

# Create empty list for storing the area counts
exp_count_list = []

# Fill the list with all the area counts
for row in test_site_exp_stats_features:
    exp_count_list.append(row['properties']['count'])

# Sum all the values and convert to area (25m2 pixel size = 0.0025 ha)
exploded_total_area_ha = sum(exp_count_list) * 0.0025

# Show the result
print("Total area (ha) for exploded geoms = ", exploded_total_area_ha)

Total area (ha) for exploded geoms =  22.915


The results are exactly the same - as hoped/expected. So the by dissolve the geometries I can get rid of the overlaps, and by exploding to single geometries I can get the zonal statistics to run without affecting the results. 

#### Additional test with Natura areas:

1) Calculate the zonal stats on the esa fnf map using the dissolve/explode approach for the Natura sites and sum up the count stats (to get the area of analysis)
2) In QGIS, calculate dissolve the Natura sites (but don't explode) and clip the sites to the CORINE footprint to get the approximate same area of analysis from step 1. Calculate the total area of this dissolved/clipped area
3) Compare results. They won't be exactly the same as in step 1 it's based of the pixel counts of the underlying raster, but in step 2 it's based on an area calculation from the vector... but I think they should be similar. 

In [2]:
# CREATE THE DISSOLVE/EXPLODE GEOMS

# Load Natura shp
natura = gpd.read_file(main_dir + "outputs/natura2000_3035_DE.shp", 
                       columns=["SITECODE", "SITENAME", "MS", "SITETYPE"])

# Dissolve all geometries to make one big polygon (with no overlaps)
big_dissolved = natura.dissolve()

# Explore multi-part polygon into individual polygons
# This is needed because the big_dissolved polygon is too big to calc zonal stats
natura_wo_overlap = big_dissolved.explode()

# Save non-overlapping geoms to shp
natura_wo_overlap.to_file(main_dir + "processing/natura_wo_overlap_test.shp")

In [5]:
# CALC ZONAL STATS FOR DISSOLVE/EXPLODE GEOMS

# Load in dissolved/exploded geoms
natura_wo_overlap = gpd.read_file(main_dir + "processing/natura_wo_overlap_test.shp")

# Load in ESA FNF (for testing)
esa_FNF = main_dir + "outputs/esa_lccs_class_3035_DE_5m_2018_FNF.tif"

# Calculate the zonal stats (count only - just need the total area)
esaFNF_natura_wo_overlaps_stats = zonal_stats(natura_wo_overlap, esa_FNF,
                                              stats=['count'],
                                              geojson_out=True)

In [ ]:
# FORMAT RESULTS

# Load as feature collection
esaFNF_natura_wo_overlaps_stats_fc = {"type": "FeatureCollection","features": esaFNF_natura_wo_overlaps_stats}

# Extract features
esa_wo_overlap_features = esaFNF_natura_wo_overlaps_stats_fc["features"]

# Check results
#esa_wo_overlap_features

In [8]:
# EXTRACT & CALC TOTAL AREA

# Create empty lists for storing the area counts
count_list = []

# Fill the list with all the area counts
for row in esa_wo_overlap_features:
    count_list.append(row['properties']['count'])

# Sum all the values and convert to area (25m2 pixel size = 0.0025 ha)
esa_wo_overlap_total_area_ha = sum(count_list) * 0.0025

# Show the result
print("Total area (ha) for exploded/dissolved geoms = ", esa_wo_overlap_total_area_ha)

Total area (ha) for exploded/dissolved geoms =  5519331.68


The results for the dissolved (but not exploded) test in QGIS was a total area of 55193.367 km2 or about 5.5 million ha. The two results are almost identical! This seems to confirm that dissolving and exploding achieves the same outcome as just dissolving - in other words, the exploding part doesn't effect the results by making the geometries too small. 

This area corresponds to the total area assessed for this thesis **without counting overlaps**.

#### Comparison:

As a final check, I compare the outputs produced using the dissolved/exploded layer without overlaps with the results for the original geometries. The percentage differences should be roughly the same as the double-counting should occur in the same areas for all 6 FNF maps. I felt this check was needed as the results were quite different from my previous results, however I had also made a fix for a classification error so I just want to confirm that this is the source of the difference (and not some issue with the overlap removal).

In [18]:
# 1.2: LOAD GDFS (ORIGINAL GEOMS)

# Load the zonal stats outputs 
esa_FNF_natura_stats_gdf = pickle.load(open(main_dir + "processing/esaFNF_natura_stats.p", "rb"))
jaxa_FNF_natura_stats_gdf = pickle.load(open(main_dir + "processing/jaxaFNF_natura_stats.p", "rb"))
corine_FNF_natura_stats_gdf = pickle.load(open(main_dir + "processing/corineFNF_natura_stats.p", "rb"))
gerlulc_FNF_natura_stats_gdf = pickle.load(open(main_dir + "processing/gerlulcFNF_natura_stats.p", "rb"))
hansen_FNF_natura_stats_gdf = pickle.load(open(main_dir + "processing/hansenFNF_natura_stats.p", "rb"))
fao_FNF_natura_stats_gdf = pickle.load(open(main_dir + "processing/faoFNF_natura_stats.p", "rb"))

In [19]:
# PER GEOMETRY CALCULATIONS (ORIGINAL GEOMS)

# Create a list of all the gdfs with the Natura 2000 zonal stats results
gdf_list = [esa_FNF_natura_stats_gdf, jaxa_FNF_natura_stats_gdf, corine_FNF_natura_stats_gdf,
            gerlulc_FNF_natura_stats_gdf, hansen_FNF_natura_stats_gdf, fao_FNF_natura_stats_gdf]

# Create new columns for count and sum in area units (hectares)
for gdf in gdf_list:
    gdf["total_area_ha"] = gdf["total_pixel_count"] * 0.0025
    gdf["total_forest_ha"] = gdf["forest_pixel_sum"] * 0.0025

# Calculate the percentage of forest in each geometry
for gdf in gdf_list:
    gdf["percent_forest"] = (gdf["total_forest_ha"] / gdf["total_area_ha"]) * 100


In [25]:
# PER FNF MAP CALCULATIONS

# Create list of names for FNF maps
fnf_map_names = ["ESA", "JAX", "COR", "DE", "H/I", "FAO"]

# Create a new partially filled dataframe for storing the results
all_fnf_natura_stats = pd.DataFrame(fnf_map_names, columns = ["Forest product"])

# Create empty lists for total forest values & total area values
forest_per_FNF = []
area_per_FNF = []

# Calculate the total forest in Natura 2000 areas for each FNF map & append to list
for gdf in gdf_list:
    total_forest = gdf["total_forest_ha"].sum()
    forest_per_FNF.append(total_forest)

    total_area = gdf["total_area_ha"].sum()
    area_per_FNF.append(total_area)    # These should all be the same results for each FNF map

# Add the total forest values and total area values to the df
all_fnf_natura_stats["Total Forest (ha)"] = forest_per_FNF
#all_fnf_natura_stats["Total Area (ha)"] = area_per_FNF

# Extract just the corine area (see notes below for why this is used for the % calc)
corine_area = area_per_FNF[2]

# Calculate the percentage of forest cover for each FNF map
all_fnf_natura_stats["Percent Forest (%)"] = (all_fnf_natura_stats["Total Forest (ha)"] / 
                                              corine_area) * 100

all_fnf_natura_stats

,Forest product,Total Forest (ha),Percent Forest (%)
0,ESA,3.870882e+06,53.407329
1,JAX,3.858510e+06,53.236626
2,COR,3.778521e+06,52.133003
3,DE,3.804294e+06,52.488603
4,H/I,3.527466e+06,48.669147
5,FAO,3.932774e+06,54.261267


In [26]:
#COMPARISON CALCULATIONS

# Calculate the average forest area across maps
avg_forest_area = all_fnf_natura_stats["Total Forest (ha)"].mean()

# Calculate the difference of each map to the average
avg_diff = all_fnf_natura_stats["Total Forest (ha)"] - avg_forest_area

# Add the difference results to the df
all_fnf_natura_stats["Difference from Mean (ha)"] = avg_diff

# Calculate the sum of each map with the average (for next step)
avg_sum = all_fnf_natura_stats["Total Forest (ha)"] + avg_forest_area

# Calculate the percent difference results to the df
percent_diff = (abs(avg_diff) / (avg_sum / 2)) * 100

# Add the percent difference results to the df
all_fnf_natura_stats["Difference from Mean (%)"] = percent_diff

all_fnf_natura_stats

,Forest product,Total Forest (ha),Percent Forest (%),Difference from Mean (ha),Difference from Mean (%)
0,ESA,3.870882e+06,53.407329,75474.240833,1.968990
1,JAX,3.858510e+06,53.236626,63101.943333,1.648880
2,COR,3.778521e+06,52.133003,-16886.991667,0.445924
3,DE,3.804294e+06,52.488603,8886.398333,0.233862
4,H/I,3.527466e+06,48.669147,-267941.959167,7.317946
5,FAO,3.932774e+06,54.261267,137366.368333,3.554947


The results from the main processing (which now uses the dissolved/exploded geometries without overlaps), the results are:

| i | map | total forest | % forest  | diff from mean | % diff   |
| - | --- | ------------ | --------- | -------------- | -------- |
| 0 | ESA | 2.872677e+06 | 52.047556 | 45599.435833   | 1.600049 |
| 1 | JAX | 2.883517e+06 | 52.243948 | 56438.958333   | 1.976640 |
| 2 | COR | 2.815114e+06 | 51.004613 | -11964.044167  | 0.424092 |
| 3 | DE  | 2.839957e+06 | 51.454730 | 12879.423333	  | 0.454538 |
| 4 | H/I | 2.632242e+06 | 47.691314 | -194835.989167 | 7.137739 |
| 5 | FAO | 2.918960e+06 | 52.886113 | 91882.215833	  | 3.198107 |

Most of the relationships between FNF maps and the % forest are similar, indicating that the main difference in these results is from the classification fix, rather than from removing the overlaps. HOWEVER one notable differences is that with overlaps included, the ESA map reports more forest than JAX, but with the overlaps removed it is the other way around (JAX>ESA)

After some thought, this difference can actually be explained by simply removing the overlaps, and perhaps highlights why even with the FNF comparison removing the double-counting is quite important! The results make sense in the following scenario:
- double-counting occurs in areas where ESA predicts more forest than JAX
- single-counting occurs in areas where JAX predicts more forest than ESA

As an example:
- In an area where Natura sites overlap, ESA predicts 100 ha of forest and JAX predicts 90 ha
- In an area where Natura sites do not overlap, ESA predicts 70 ha of forest and JAX predicts 100 ha
- When **overlaps are included**, ESA would predict a total of 270 ha of forest (100+100+70) and JAX would predict 250 ha (90+90+70) - in other words: **ESA predicts more forest than JAX.**
- When **overlaps are removed**, ESA would predict 170 ha of forest (100+70) and JAX would predict 190 ha (90+100) - so then in this case, **JAX predicts more forest than ESA.**

So, I think the approach to remove the overlaps still makes sense and the scenario described above highlights something which I had not considered previously, and demonstrates the importance of removing the overlaps when wanting to understand the differences between maps. 